Step 1: Data Cleaning and Saving Cleaned Versions

In [1]:
import pandas as pd
import numpy as np

creditcard_df = pd.read_csv('../data/creditcard.csv')
fraud_data_df = pd.read_csv('../data/Fraud_Data.csv')

In [2]:
creditcard_df.drop_duplicates(inplace=True)

In [3]:
creditcard_df['Time'] = pd.to_datetime(creditcard_df['Time'], unit='s', origin='2010-01-01')

In [4]:
creditcard_df.to_csv('../data/cleaned_creditcard.csv', index=False)

In [5]:
fraud_data_df.drop_duplicates(inplace=True)

In [6]:
fraud_data_df['signup_time'] = pd.to_datetime(fraud_data_df['signup_time'])
fraud_data_df['purchase_time'] = pd.to_datetime(fraud_data_df['purchase_time'])

In [7]:
fraud_data_df.to_csv('../data/cleaned_fraud_data.csv', index=False)

Feature and Target Separation

In [8]:
X_credit = creditcard_df.drop(columns=['Class'])
y_credit = creditcard_df['Class']

X_fraud = fraud_data_df.drop(columns=['class'])
y_fraud = fraud_data_df['class']

Train-Test Split

In [9]:
from sklearn.model_selection import train_test_split

X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit)

X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)

Step 2: Model Selection

In [16]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical

mlflow.set_experiment('Fraud Detection Models')

def log_mlflow_model(model, model_name, X_train, X_test, y_train, y_test):
    with mlflow.start_run(run_name=model_name):
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        
        mlflow.log_param('model_name', model_name)
        mlflow.log_metrics({
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc
        })
        
        
        mlflow.sklearn.log_model(model, model_name)
        
        print(f"Model {model_name}: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}")

In [17]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'MLP': MLPClassifier(random_state=42, max_iter=1000)
}